In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Vamos abrir os dados de teste do UCI-HAR
pasta = "C:\\Users\\Usuario\\Desktop\\UCI HAR Dataset\\UCI HAR Dataset\\test\\"
dados = [None]*6
dados[0] = np.loadtxt(pasta+'Inertial Signals\\body_acc_x_test.txt')
dados[1] = np.loadtxt(pasta+'Inertial Signals\\body_acc_y_test.txt')
dados[2] = np.loadtxt(pasta+'Inertial Signals\\body_acc_z_test.txt')
dados[3] = np.loadtxt(pasta+'Inertial Signals\\body_gyro_x_test.txt')
dados[4] = np.loadtxt(pasta+'Inertial Signals\\body_gyro_y_test.txt')
dados[5] = np.loadtxt(pasta+'Inertial Signals\\body_gyro_z_test.txt')
y = np.loadtxt(pasta+'y_test.txt')
user = np.loadtxt(pasta+'subject_test.txt')
labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING']
print('Número de amostras por classe \npara janelas de 2,56 segundos e sobreposição de 50%')
for i in np.unique(y):
    print(labels[i.astype(int)-1]+': '+str(np.sum(y==i)))

Número de amostras por classe 
para janelas de 2,56 segundos e sobreposição de 50%
WALKING: 496
WALKING_UPSTAIRS: 471
WALKING_DOWNSTAIRS: 420
SITTING: 491
STANDING: 532
LAYING: 537


In [3]:
X = dados[0]

In [4]:
# Vamos observar a sobreposição das janelas
fig = go.Figure()

for i in range(20):
    t = np.arange(i*64,i*64+128)/50
    fig.add_trace(go.Scatter(x=t, y=X[i,:], mode='lines', showlegend=True))

fig.show()

In [5]:
# Vamos ver quando ocorre uma nova amostra (ou seja, quando não ocorre sobreposição)
aux = np.zeros(len(y)-1)
for i in range(len(y)-1):
    aux[i] = np.all(X[i,64:]!=X[i+1,:64])

# Agora vamos encontrar os índices de cada nova amostra
ind = np.nonzero(aux)[0]+1
ind = np.append([0],ind)
ind = np.append(ind,[len(y)])

In [6]:
# Agora vamos construir as capturas originais (sem sobreposição)
sigs = []
for i in range(len(ind)-1):
    sig = X[ind[i]:ind[i+1],64:].reshape((ind[i+1]-ind[i])*64)
    sig = np.append(X[ind[i],:64],sig)
    sigs.append(sig)

In [7]:
# Vamos construir as janelas de 3 segundos (lembrando que o UCI-HAR está amostrado a uma taxa de 50 Hz)
aux = np.zeros(len(sigs))
for i in range(len(sigs)):
    aux[i] = len(sigs[i])
Na = np.sum(aux//150, dtype=np.int64)
mat = np.zeros((Na,150))
k = 0
for i in range(len(sigs)):
    for j in range(len(sigs[i])//150):
        mat[k,:] = sigs[i][j*150:(j+1)*150]
        k+=1

In [8]:
def transformar(X, y, user):
    """Essa função é responsável por pegar os dados do UCI-HAR
    (recortados em janelas de 2,56 segundos e com sobreposição de 50%)
    e transformá-los em janelas de 3 segundos sem sobreposição"""
    aux = np.zeros(len(y)-1)
    for i in range(len(y)-1):
        aux[i] = np.all(X[i,64:]!=X[i+1,:64])
    ind = np.nonzero(aux)[0]+1
    ind = np.append([0],ind)
    ind = np.append(ind,[len(y)])
    sigs = []
    yy = []
    uuser = []
    for i in range(len(ind)-1):
        sig = X[ind[i]:ind[i+1],64:].reshape((ind[i+1]-ind[i])*64)
        sig = np.append(X[ind[i],:64],sig)
        sigs.append(sig)
        yy.append(y[ind[i]])
        uuser.append(user[ind[i]])
    
    aux = np.zeros(len(sigs))
    for i in range(len(sigs)):
        aux[i] = len(sigs[i])
    Na = np.sum(aux//150, dtype=np.int64)
    mat = np.zeros((Na,150))
    yyy = np.zeros(Na)
    uuuser = np.zeros(Na)
    k = 0
    for i in range(len(sigs)):
        for j in range(len(sigs[i])//150):
            mat[k,:] = sigs[i][j*150:(j+1)*150]
            yyy[k] = yy[i]
            uuuser[k] = uuser[i]
            k+=1
    return mat, yyy, uuuser;

In [9]:
test = np.zeros((Na,150*6))
for i in range(6):
    mat, yyy, uuuser = transformar(dados[i], y, user)
    test[:,i*150:(i+1)*150] = mat
test = np.column_stack((test,yyy.astype(int),uuuser.astype(int)))

In [10]:
df = pd.DataFrame(data=test)

In [11]:
colunas = ['accel-x-' + str(x) for x in np.arange(150).tolist()]
colunas += ['accel-y-' + str(x) for x in np.arange(150).tolist()]
colunas += ['accel-z-' + str(x) for x in np.arange(150).tolist()]
colunas += ['gyro-x-' + str(x) for x in np.arange(150).tolist()]
colunas += ['gyro-y-' + str(x) for x in np.arange(150).tolist()]
colunas += ['gyro-z-' + str(x) for x in np.arange(150).tolist()]
colunas += ['activity']
colunas += ['user']
df.columns = colunas
df = df.astype({"activity": int, "user": int})

In [12]:
print('Número de amostras por classe na formatação final:')
for i in np.unique(yyy):
    print(labels[i.astype(int)-1]+': '+str(np.sum(yyy==i)))

Número de amostras por classe na formatação final:
WALKING: 204
WALKING_UPSTAIRS: 189
WALKING_DOWNSTAIRS: 173
SITTING: 204
STANDING: 227
LAYING: 227


In [13]:
df.to_csv('UCIHAR_test.csv',index=False)

In [15]:
# Agora vamos abrir os dados de treino do UCI-HAR
pasta = "C:\\Users\\Usuario\\Desktop\\UCI HAR Dataset\\UCI HAR Dataset\\train\\"
dados = [None]*6
dados[0] = np.loadtxt(pasta+'Inertial Signals\\body_acc_x_train.txt')
dados[1] = np.loadtxt(pasta+'Inertial Signals\\body_acc_y_train.txt')
dados[2] = np.loadtxt(pasta+'Inertial Signals\\body_acc_z_train.txt')
dados[3] = np.loadtxt(pasta+'Inertial Signals\\body_gyro_x_train.txt')
dados[4] = np.loadtxt(pasta+'Inertial Signals\\body_gyro_y_train.txt')
dados[5] = np.loadtxt(pasta+'Inertial Signals\\body_gyro_z_train.txt')
y = np.loadtxt(pasta+'y_train.txt')
user = np.loadtxt(pasta+'subject_train.txt')
labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING']
print('Número de amostras por classe \npara janelas de 2,56 segundos e sobreposição de 50%')
for i in np.unique(y):
    print(labels[i.astype(int)-1]+': '+str(np.sum(y==i)))

Número de amostras por classe 
para janelas de 2,56 segundos e sobreposição de 50%
WALKING: 1226
WALKING_UPSTAIRS: 1073
WALKING_DOWNSTAIRS: 986
SITTING: 1286
STANDING: 1374
LAYING: 1407


In [16]:
X = dados[0]

In [17]:
# Vamos ver quando ocorre uma nova amostra (ou seja, quando não ocorre sobreposição)
aux = np.zeros(len(y)-1)
for i in range(len(y)-1):
    aux[i] = np.all(X[i,64:]!=X[i+1,:64])

# Agora vamos encontrar os índices de cada nova amostra
ind = np.nonzero(aux)[0]+1
ind = np.append([0],ind)
ind = np.append(ind,[len(y)])

In [18]:
# Agora vamos construir as capturas originais (sem sobreposição)
sigs = []
for i in range(len(ind)-1):
    sig = X[ind[i]:ind[i+1],64:].reshape((ind[i+1]-ind[i])*64)
    sig = np.append(X[ind[i],:64],sig)
    sigs.append(sig)

In [19]:
# Vamos construir as janelas de 3 segundos (lembrando que o UCI-HAR está amostrado a uma taxa de 50 Hz)
aux = np.zeros(len(sigs))
for i in range(len(sigs)):
    aux[i] = len(sigs[i])
Na = np.sum(aux//150, dtype=np.int64)
mat = np.zeros((Na,150))
k = 0
for i in range(len(sigs)):
    for j in range(len(sigs[i])//150):
        mat[k,:] = sigs[i][j*150:(j+1)*150]
        k+=1

In [20]:
train = np.zeros((Na,150*6))
for i in range(6):
    mat, yyy, uuuser = transformar(dados[i], y, user)
    train[:,i*150:(i+1)*150] = mat
train = np.column_stack((train,yyy.astype(int),uuuser.astype(int)))

In [21]:
df = pd.DataFrame(data=train)

In [22]:
colunas = ['accel-x-' + str(x) for x in np.arange(150).tolist()]
colunas += ['accel-y-' + str(x) for x in np.arange(150).tolist()]
colunas += ['accel-z-' + str(x) for x in np.arange(150).tolist()]
colunas += ['gyro-x-' + str(x) for x in np.arange(150).tolist()]
colunas += ['gyro-y-' + str(x) for x in np.arange(150).tolist()]
colunas += ['gyro-z-' + str(x) for x in np.arange(150).tolist()]
colunas += ['activity']
colunas += ['user']
df.columns = colunas
df = df.astype({"activity": int, "user": int})

In [23]:
print('Número de amostras por classe na formatação final:')
for i in np.unique(yyy):
    print(labels[i.astype(int)-1]+': '+str(np.sum(yyy==i)))

Número de amostras por classe na formatação final:
WALKING: 506
WALKING_UPSTAIRS: 439
WALKING_DOWNSTAIRS: 395
SITTING: 544
STANDING: 575
LAYING: 590


In [ ]:
df.to_csv('UCIHAR_train.csv',index=False)